In [2]:
import os
import clang
from clang.cindex import *
from copy import deepcopy

In [10]:
! wget https://zenodo.org/record/3628775/files/c-corpus.tar.gz?download=1
! tar -xzf "/kaggle/working/c-corpus.tar.gz?download=1"
! rm /kaggle/working/c-corpus.tar.gz?download=1

In [63]:
! sudo apt-get update
! sudo apt update
! sudo apt-get install clang -y
! sudo apt-get install libclang-dev -y
! sudo apt-get install python3-clang -y

Config.set_library_file("/usr/lib/x86_64-linux-gnu/libclang-10.so")

find: ‘libclang-10.so’: No such file or directory


In [67]:
# create an index and parse the source code
index = Index.create()
root_cursor = index.parse("/kaggle/working/cleaned/e2openplugin-crossepg/src/common/crossepg_exporter.c").cursor
# root_cursor = index.parse("/home/dipu/Documents/AI/MinorProject/c-corpus/cleaned/gno/lib/libc/tests/sys/rename.c").cursor

In [68]:
# def print_ast(node, indent):
#     try:
#         print(" "*indent + node.spelling + " " + str(node.kind) + " " + str(node.type.spelling))
#         for c in node.get_children():
#             print_ast(c, indent+2)
#     except ValueError:
#         pass

# print_ast(root_cursor, 0)

In [69]:
def get_function_params(node, function_name, result):
    try:
        if node.kind == CursorKind.FUNCTION_DECL and node.spelling == function_name:
            for c in node.get_children():
                if c.kind == CursorKind.PARM_DECL:
                    # print(c.spelling + " " + str(c.kind) + " " + str(c.type.spelling))
                    result.append({"name": c.spelling, "data_type": c.type.spelling})
            return
    except ValueError:
        pass

    for c in node.get_children():
        get_function_params(c, function_name, result)

In [70]:
param_list = []
get_function_params(root_cursor, "getSum", param_list)
param_list

[]

In [71]:
def get_called_functions(root, node, result):
    try:
        if node.kind == CursorKind.CALL_EXPR:
            # "location": node.extent
            current_function = {"name": node.spelling, "return_type": node.type.spelling, "args": []}
            for c in node.get_arguments():
                current_function["args"].append({"name": list(c.get_tokens())[0].spelling if len(list(c.get_tokens())) > 0 else c.spelling, "data_type": c.type.spelling, "cursor_kind": c.kind})
                # current_function["args"].append({"name": c.spelling, "data_type": c.type.spelling, "cursor_kind": c.kind})
                # print(node.location)
                
            current_param_list = []
            get_function_params(root, node.spelling, current_param_list)
            current_function["params"] = current_param_list

            result.append(current_function)
            return
    except ValueError:
        pass

    for c in node.get_children():
        get_called_functions(root, c, result)

In [72]:
function_list = []
get_called_functions(root_cursor, root_cursor, function_list)
function_list

[{'name': 'printf',
  'return_type': 'int',
  'args': [{'name': '"Usage:\\n"',
    'data_type': 'const char *',
    'cursor_kind': CursorKind.UNEXPOSED_EXPR}],
  'params': [{'name': '__format', 'data_type': 'const char *restrict'}]},
 {'name': 'printf',
  'return_type': 'int',
  'args': [{'name': '"  ./crossepg_exporter [options] output_file\\n"',
    'data_type': 'const char *',
    'cursor_kind': CursorKind.UNEXPOSED_EXPR}],
  'params': [{'name': '__format', 'data_type': 'const char *restrict'}]},
 {'name': 'printf',
  'return_type': 'int',
  'args': [{'name': '"Options:\\n"',
    'data_type': 'const char *',
    'cursor_kind': CursorKind.UNEXPOSED_EXPR}],
  'params': [{'name': '__format', 'data_type': 'const char *restrict'}]},
 {'name': 'printf',
  'return_type': 'int',
  'args': [{'name': '"  -d db_root       crossepg db root folder\\n"',
    'data_type': 'const char *',
    'cursor_kind': CursorKind.UNEXPOSED_EXPR}],
  'params': [{'name': '__format', 'data_type': 'const char *res

In [73]:
for function in function_list:
        
        if len(function["args"]) == 2 and \
            (function["args"][0]["data_type"] == function["args"][1]["data_type"]) and \
            (function["args"][0]["name"] != function["args"][1]["name"]):
            
            positive_sample = [function["name"], function["args"][0]["name"], function["args"][1]["name"],
                              function["return_type"]]
            
            if(len(function["params"]) == 2):
                positive_sample.append(function["params"][0]["name"])
                positive_sample.append(function["params"][1]["name"])
            else:
                positive_sample.append("")
                positive_sample.append("")
            
            negative_sample = deepcopy(positive_sample)
            # swap
            negative_sample[1], negative_sample[2] = negative_sample[2], negative_sample[1]
            
            positive_sample.append(str(0))
            negative_sample.append(str(1))
            
            positive_sample = ",".join(positive_sample)
            negative_sample = ",".join(negative_sample)
            
            print(positive_sample)
            print(negative_sample)
            

log_add,"EPGDB opened (root=%s)",db_root,int,,,0
log_add,db_root,"EPGDB opened (root=%s)",int,,,1
log_add,"Exporting data into %s",outputfile,int,,,0
log_add,outputfile,"Exporting data into %s",int,,,1
fopen,outputfile,"w",FILE *,__filename,__modes,0
fopen,"w",outputfile,FILE *,__filename,__modes,1
log_add,"Cannot open %s",outputfile,int,,,0
log_add,outputfile,"Cannot open %s",int,,,1


---------
## Negative sample generation from all code snippets - Function args swap only
---------

In [77]:
def generate_function_args_swap_dataset(root_dir):
    total_files, total_samples = 0, 0
    global current_file
    
    with open("function_swap_samples.csv", 'a') as function_swap_samples:
        function_swap_samples.write("function_name,arg1,arg2,arg_type,param1,param2,labels")
        
        for root, dirs, files in os.walk(root_dir):
                for file in files:
                    if file.endswith(".c"):
                        total_files += 1

                        file_path = os.path.join(root, file)
                        with open(file_path, 'rb') as f:
                            # content = str(f.read())
                            start_cursor = index.parse(f.name).cursor

                            function_list = []
                            get_called_functions(start_cursor, start_cursor, function_list)

                            for function in function_list:

                                    if len(function["args"]) == 2 and \
                                        (function["args"][0]["data_type"] == function["args"][1]["data_type"]) and \
                                        (function["args"][0]["name"] != function["args"][1]["name"]):

                                        positive_sample = [function["name"], function["args"][0]["name"], function["args"][1]["name"],
                                                          function["return_type"]]

                                        if(len(function["params"]) == 2):
                                            positive_sample.append(function["params"][0]["name"])
                                            positive_sample.append(function["params"][1]["name"])
                                        else:
                                            positive_sample.append("")
                                            positive_sample.append("")

                                        negative_sample = deepcopy(positive_sample)
                                        # swap
                                        negative_sample[1], negative_sample[2] = negative_sample[2], negative_sample[1]

                                        positive_sample.append(str(0))
                                        negative_sample.append(str(1))

                                        positive_sample = ",".join(positive_sample)
                                        negative_sample = ",".join(negative_sample)
                                        
                                        function_swap_samples.write("\n" + positive_sample)
                                        function_swap_samples.write("\n" + negative_sample)
                                        
                                        total_samples += 1
                                        
                            current_file = f.name

                    if total_files % 1000 == 0:
                        print("Total files:", total_files, ",", "Total samples:", total_samples)

In [79]:
root_dir = '/kaggle/working/cleaned'

generate_function_args_swap_dataset(root_dir)

Total files: 100 , Total samples: 161
Total files: 200 , Total samples: 167
Total files: 300 , Total samples: 189
Total files: 400 , Total samples: 194
Total files: 500 , Total samples: 231


KeyboardInterrupt: 

In [80]:
# current_file